## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Mar. 9th Mar. 16th


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,March-14-2025,Condo/Co-op,17975 Collins Ave Unit 4701/2,Sunny Isles Beach,FL,33160.0,25000000.0,6.0,7.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11764383,N,Y,25.942332,-80.120166


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df2 = df[(df['SOLD DATE'] > one_week_ago) & (df['SOLD DATE'] < today)]

In [10]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-03-10    42
2025-03-12    28
2025-03-14    24
2025-03-11    21
2025-03-13    20
Name: count, dtype: int64

In [11]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [12]:
df_top_ten = df2.head(10)

In [13]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,2025-03-14,Condo/Co-op,17975 Collins Ave Unit 4701/2,Sunny Isles Beach,FL,33160.0,25000000.0,6.0,7.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11764383,N,Y,25.942332,-80.120166
4,PAST SALE,2025-03-11,Condo/Co-op,125 Ocean Dr,Miami Beach,FL,33139.0,24000000.0,4.0,5.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/125-Ocea...,MARMLS,A11738003,N,Y,25.770080,-80.132744
108,PAST SALE,2025-03-11,Condo/Co-op,7264 Fisher Is #7264,Fisher Island,FL,33109.0,10900000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7264-Fis...,MARMLS,A11653700,N,Y,25.762287,-80.134708
134,PAST SALE,2025-03-12,Condo/Co-op,18501 Collins Ave #801,Sunny Isles Beach,FL,33160.0,6000000.0,4.0,5.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Beaches MLS,RX-10968656,N,Y,25.947720,-80.120228
80,PAST SALE,2025-03-10,Condo/Co-op,15701 Collins Ave #3602,Sunny Isles Beach,FL,33160.0,5445000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11690661,N,Y,25.921944,-80.121716
137,PAST SALE,2025-03-11,Condo/Co-op,6800 Indian Creek Dr Unit 7A,Miami Beach,FL,33141.0,4800000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6800-Ind...,MARMLS,A11471887,N,Y,25.852444,-80.122990
63,PAST SALE,2025-03-10,Condo/Co-op,6000 Island Blvd #2604,Aventura,FL,33160.0,4775000.0,5.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/6000-Island...,MARMLS,A11699721,N,Y,25.942761,-80.135433
62,PAST SALE,2025-03-10,Condo/Co-op,17875 Collins Ave #4401,Sunny Isles Beach,FL,33160.0,4350000.0,4.0,4.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11699457,N,Y,25.941257,-80.120772
110,PAST SALE,2025-03-12,Condo/Co-op,16901 Collins Ave #2602,Sunny Isles Beach,FL,33160.0,4000000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11651859,N,Y,25.932764,-80.121036
133,PAST SALE,2025-03-13,Condo/Co-op,50 S Pointe Dr #1207,Miami Beach,FL,33139.0,3500000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/50-S-Poi...,MARMLS,A11553824,N,Y,25.767875,-80.132756


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [14]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [15]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [16]:
response_df = pd.DataFrame(response_list)

In [17]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [18]:
df_top_ten = merged_df

In [19]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-03-14,Condo/Co-op,17975 Collins Ave Unit 4701/2,Sunny Isles Beach,FL,33160.0,25000000.0,6.0,7.0,...,-80.120166,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Michael Goldstein,Acqualina Realty,None,None,Benjamin Moss,"Compass Florida, LLC.",Liz Hogan,"Compass Florida, LLC."
1,PAST SALE,2025-03-11,Condo/Co-op,125 Ocean Dr,Miami Beach,FL,33139.0,24000000.0,4.0,5.0,...,-80.132744,https://www.redfin.com/FL/Miami-Beach/125-Ocea...,Sarah Williams,Societe Real Estate,None,None,Michael Duchon,The Corcoran Group,Eloy Carmenate,The Corcoran Group
2,PAST SALE,2025-03-11,Condo/Co-op,7264 Fisher Is #7264,Fisher Island,FL,33109.0,10900000.0,3.0,3.5,...,-80.134708,https://www.redfin.com/FL/Miami-Beach/7264-Fis...,Lazaro Rivero,"Rivero Real Estate , Inc.",Rose Bauer,"Rivero Real Estate , Inc.",Lazaro Rivero,"Rivero Real Estate , Inc.",None,None
3,PAST SALE,2025-03-12,Condo/Co-op,18501 Collins Ave #801,Sunny Isles Beach,FL,33160.0,6000000.0,4.0,5.5,...,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Heidi Brzyski,Douglas Elliman (Jupiter),Tracy Servideo,Douglas Elliman (Jupiter),Julio Oliveira,Elite International Realty Inc,None,None
4,PAST SALE,2025-03-10,Condo/Co-op,15701 Collins Ave #3602,Sunny Isles Beach,FL,33160.0,5445000.0,3.0,3.5,...,-80.121716,https://www.redfin.com/FL/North-Miami-Beach/15...,Kelly Salinas,RE/MAX Advance Realty II,None,None,Dennis Mangone,Douglas Elliman,None,None
5,PAST SALE,2025-03-11,Condo/Co-op,6800 Indian Creek Dr Unit 7A,Miami Beach,FL,33141.0,4800000.0,3.0,3.5,...,-80.122990,https://www.redfin.com/FL/Miami-Beach/6800-Ind...,Nelson Gonzalez,BHHS EWM Realty,None,None,Arno De Vos,De Vos Real Estate,Paul Basile,Douglas Elliman
6,PAST SALE,2025-03-10,Condo/Co-op,6000 Island Blvd #2604,Aventura,FL,33160.0,4775000.0,5.0,4.5,...,-80.135433,https://www.redfin.com/FL/Aventura/6000-Island...,Richard Goihman,Douglas Elliman,Ivonne Goihman,Douglas Elliman,Richard Goihman,Douglas Elliman,None,None
7,PAST SALE,2025-03-10,Condo/Co-op,17875 Collins Ave #4401,Sunny Isles Beach,FL,33160.0,4350000.0,4.0,4.0,...,-80.120772,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Yansy Checa,Douglas Elliman,None,None,Monica Villate,EXP Realty LLC,None,None
8,PAST SALE,2025-03-12,Condo/Co-op,16901 Collins Ave #2602,Sunny Isles Beach,FL,33160.0,4000000.0,3.0,4.5,...,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Rosa Maria Pinneri,LPT Realty,Kinga Konsorska,Douglas Elliman,Wictor Arraes,Xcellence Realty,None,None
9,PAST SALE,2025-03-13,Condo/Co-op,50 S Pointe Dr #1207,Miami Beach,FL,33139.0,3500000.0,2.0,2.5,...,-80.132756,https://www.redfin.com/FL/Miami-Beach/50-S-Poi...,Lisa Yanowitz,Luxe Realty Group LLC,None,None,Sonia Toth,Brown Harris Stevens,None,None


## Current Week's Values

In [20]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

135


In [21]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$1,312,551


In [22]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$685


In [23]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$177,194,410


In [24]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [26]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input county name:
Miami-Dade
Input Previous Week Condo Sales Total (number units sold):
117
-----------
Input Previous Week Condo Average Sales Price:
1032863
-----------
Input Previous Week Condo Average PSF:
653
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
120844976.0


In [27]:
last_week

,Condo Sales,Average Sale Price,Average PSF,Last Week Total Dollar Volume
0,117,1032863,653,120844976.0


In [28]:
last_week['Last Week Total Dollar Volume'].iloc[0]

120844976.0

In [29]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0          135             1312551          685                    177194410.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [30]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [31]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [32]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [33]:
pd.set_option('display.max_columns',None)

In [34]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-03-14,Condo/Co-op,17975 Collins Ave Unit 4701/2,Sunny Isles Beach,FL,33160.0,25000000.0,6.0,7.0,ESTATES AT ACQUALINA,9100.0,NaN,2024.0,NaN,2747.0,34638.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11764383,N,Y,25.942332,-80.120166,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Michael Goldstein,Acqualina Realty,None,None,Benjamin Moss,"Compass Florida, LLC.",Liz Hogan,"Compass Florida, LLC.",1,orange


In [35]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [36]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-03-14,Condo/Co-op,17975 Collins Ave Unit 4701/2,Sunny Isles Beach,FL,33160.0,25000000.0,6.0,7.0,ESTATES AT ACQUALINA,9100.0,NaN,2024.0,NaN,2747.0,34638.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11764383,N,Y,25.942332,-80.120166,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Michael Goldstein,Acqualina Realty,None,None,Benjamin Moss,"Compass Florida, LLC.",Liz Hogan,"Compass Florida, LLC.",1,orange
1,PAST SALE,2025-03-11,Condo/Co-op,125 Ocean Dr,Miami Beach,FL,33139.0,24000000.0,4.0,5.0,OCEAN HOUSE CONDO,3936.0,NaN,2009.0,NaN,6098.0,10001.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/125-Ocea...,MARMLS,A11738003,N,Y,25.770080,-80.132744,https://www.redfin.com/FL/Miami-Beach/125-Ocea...,Sarah Williams,Societe Real Estate,None,None,Michael Duchon,The Corcoran Group,Eloy Carmenate,The Corcoran Group,2,blue
2,PAST SALE,2025-03-11,Condo/Co-op,7264 Fisher Is #7264,Fisher Island,FL,33109.0,10900000.0,3.0,3.5,Oceanside,4590.0,NaN,1992.0,NaN,2375.0,9738.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7264-Fis...,MARMLS,A11653700,N,Y,25.762287,-80.134708,https://www.redfin.com/FL/Miami-Beach/7264-Fis...,Lazaro Rivero,"Rivero Real Estate , Inc.",Rose Bauer,"Rivero Real Estate , Inc.",Lazaro Rivero,"Rivero Real Estate , Inc.",None,None,3,blue
3,PAST SALE,2025-03-12,Condo/Co-op,18501 Collins Ave #801,Sunny Isles Beach,FL,33160.0,6000000.0,4.0,5.5,Turnberry Ocean Club,3252.0,NaN,2020.0,NaN,1845.0,6063.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Beaches MLS,RX-10968656,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Heidi Brzyski,Douglas Elliman (Jupiter),Tracy Servideo,Douglas Elliman (Jupiter),Julio Oliveira,Elite International Realty Inc,None,None,4,blue
4,PAST SALE,2025-03-10,Condo/Co-op,15701 Collins Ave #3602,Sunny Isles Beach,FL,33160.0,5445000.0,3.0,3.5,15701 COLLINS CONDO,2475.0,NaN,2020.0,NaN,2200.0,3640.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11690661,N,Y,25.921944,-80.121716,https://www.redfin.com/FL/North-Miami-Beach/15...,Kelly Salinas,RE/MAX Advance Realty II,None,None,Dennis Mangone,Douglas Elliman,None,None,5,blue
5,PAST SALE,2025-03-11,Condo/Co-op,6800 Indian Creek Dr Unit 7A,Miami Beach,FL,33141.0,4800000.0,3.0,3.5,AMD PL OF 2ND OCEAN FRONT,2844.0,NaN,2022.0,NaN,1688.0,6414.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6800-Ind...,MARMLS,A11471887,N,Y,25.852444,-80.122990,https://www.redfin.com/FL/Miami-Beach/6800-Ind...,Nelson Gonzalez,BHHS EWM Realty,None,None,Arno De Vos,De Vos Real Estate,Paul Basile,Douglas Elliman,6,blue
6,PAST SALE,2025-03-10,Condo/Co-op,6000 Island Blvd #2604,Aventura,FL,33160.0,4775000.0,5.0,4.5,BELLAMARE @ WILLIAMS ISLA,4221.0,NaN,2005.0,NaN,1131.0,4608.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/6000-Island...,MARMLS,A11699721,N,Y,25.942761,-80.135433,https://www.redfin.com/FL/Aventura/6000-Island...,Richard Goihman,Douglas Elliman,Ivonne Goihman,Douglas Elliman,Richard Goihman,Douglas Elliman,None,None,7,blue
7,PAST SALE,2025-03-10,Condo/Co-op,17875 Collins Ave #4401,Sunny Isles Beach,FL,33160.0,4350000.0,4.0,4.0,ACQUALINA OCEAN RESIDENCE,2736.0,NaN,2006.0,NaN,1590.0,5095.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11699457,N,Y,25.941257,-80.120772,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Yansy Checa,Douglas Elliman,None,None,Monica Villate,EXP Realty LLC,None,None,8,blue
8,PAST SALE,2025-03-12,Condo/Co-op,16901 Collins Ave #2602,Sunny Isles Beach,FL,33160.0

In [37]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [38]:
m.save('index.html')

## Data snagger

In [39]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [40]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-03-14,Condo/Co-op,17975 Collins Ave Unit 4701/2,Sunny Isles Beach,FL,33160.0,25000000.0,6.0,7.0,ESTATES AT ACQUALINA,9100.0,NaN,2024.0,NaN,2747.0,34638.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11764383,N,Y,25.942332,-80.120166,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Michael Goldstein,Acqualina Realty,None,None,Benjamin Moss,"Compass Florida, LLC.",Liz Hogan,"Compass Florida, LLC.",1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [41]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_03172024


# CREATE TEMPLATE 

In [42]:
muni_set = set(df_top_ten['CITY'])

In [43]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [44]:
df_top_ten.reset_index(inplace=True,drop=True)

In [45]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [46]:
top_sale

'ESTATES AT ACQUALINA at 17975 Collins Ave Unit 4701/2 in Sunny Isles Beach'

In [47]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [48]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [49]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-03-14,Condo/Co-op,17975 Collins Ave Unit 4701/2,Sunny Isles Beach,FL,33160.0,25000000.0,6.0,7.0,ESTATES AT ACQUALINA,9100.0,NaN,2024.0,NaN,2747.0,34638.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11764383,N,Y,25.942332,-80.120166,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Michael Goldstein,Acqualina Realty,None,None,Benjamin Moss,"Compass Florida, LLC.",Liz Hogan,"Compass Florida, LLC.",1,orange,ESTATES AT ACQUALINA at 17975 Collins Ave Unit...
1,PAST SALE,2025-03-11,Condo/Co-op,125 Ocean Dr,Miami Beach,FL,33139.0,24000000.0,4.0,5.0,OCEAN HOUSE CONDO,3936.0,NaN,2009.0,NaN,6098.0,10001.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/125-Ocea...,MARMLS,A11738003,N,Y,25.770080,-80.132744,https://www.redfin.com/FL/Miami-Beach/125-Ocea...,Sarah Williams,Societe Real Estate,None,None,Michael Duchon,The Corcoran Group,Eloy Carmenate,The Corcoran Group,2,blue,OCEAN HOUSE CONDO at 125 Ocean Dr in Miami Beach
2,PAST SALE,2025-03-11,Condo/Co-op,7264 Fisher Is #7264,Fisher Island,FL,33109.0,10900000.0,3.0,3.5,Oceanside,4590.0,NaN,1992.0,NaN,2375.0,9738.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7264-Fis...,MARMLS,A11653700,N,Y,25.762287,-80.134708,https://www.redfin.com/FL/Miami-Beach/7264-Fis...,Lazaro Rivero,"Rivero Real Estate , Inc.",Rose Bauer,"Rivero Real Estate , Inc.",Lazaro Rivero,"Rivero Real Estate , Inc.",None,None,3,blue,Oceanside at 7264 Fisher Is #7264 in Fisher Is...
3,PAST SALE,2025-03-12,Condo/Co-op,18501 Collins Ave #801,Sunny Isles Beach,FL,33160.0,6000000.0,4.0,5.5,Turnberry Ocean Club,3252.0,NaN,2020.0,NaN,1845.0,6063.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Beaches MLS,RX-10968656,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Heidi Brzyski,Douglas Elliman (Jupiter),Tracy Servideo,Douglas Elliman (Jupiter),Julio Oliveira,Elite International Realty Inc,None,None,4,blue,Turnberry Ocean Club at 18501 Collins Ave #801...
4,PAST SALE,2025-03-10,Condo/Co-op,15701 Collins Ave #3602,Sunny Isles Beach,FL,33160.0,5445000.0,3.0,3.5,15701 COLLINS CONDO,2475.0,NaN,2020.0,NaN,2200.0,3640.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11690661,N,Y,25.921944,-80.121716,https://www.redfin.com/FL/North-Miami-Beach/15...,Kelly Salinas,RE/MAX Advance Realty II,None,None,Dennis Mangone,Douglas Elliman,None,None,5,blue,15701 COLLINS CONDO at 15701 Collins Ave #3602...
5,PAST SALE,2025-03-11,Condo/Co-op,6800 Indian Creek Dr Unit 7A,Miami Beach,FL,33141.0,4800000.0,3.0,3.5,AMD PL OF 2ND OCEAN FRONT,2844.0,NaN,2022.0,NaN,1688.0,6414.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6800-Ind...,MARMLS,A11471887,N,Y,25.852444,-80.122990,https://www.redfin.com/FL/Miami-Beach/6800-Ind...,Nelson Gonzalez,BHHS EWM Realty,None,None,Arno De Vos,De Vos Real Estate,Paul Basile,Douglas Elliman,6,blue,AMD PL OF 2ND OCEAN FRONT at 6800 Indian Creek...
6,PAST SALE,2025-03-10,Condo/Co-op,6000 Island Blvd #2604,Aventura,FL,33160.0,4775000.0,5.0,4.5,BELLAMARE @ WILLIAMS ISLA,4221.0,NaN,2005.0,NaN,1131.0,4608.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/6000-Island...,MARMLS,A11699721,N,Y,25.942761,-80.135433,https://www.redfin.com/FL/Aventura/6000-Island...,Richard Goihman,Douglas Elliman,Ivonne Goihman,Douglas Elliman,Richard Goihman,Douglas Elliman,None,None,7,blue,BELLAMARE @ WILLIAMS ISLA at 6000 Island Blvd ...
7,PAST SALE,2025-03-10,Condo/Co-op,17875 Collins Ave #4401,Sunny Isles Beach,FL,33160.0,4350000.0,4.0,4.0,ACQUALINA OCEAN RESIDENCE,2736

In [50]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: ESTATES AT ACQUALINA closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $3,500,000 to $25,000,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami Beach, Fisher Island, Sunny Isles Beach, Aventura, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 135 condo sales totaling $177,194,410 million from Mar. 9th to Mar. 16th. The previous week, brokers closed 117 condo sales totaling $120,844,976.

Last week’s units sold for an average of $1,312,551, higher than the $1,032,863 average pri

In [51]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [52]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [53]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [54]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-03-14,Condo/Co-op,17975 Collins Ave Unit 4701/2,Sunny Isles Beach,FL,33160.0,25000000.0,6.0,7.0,ESTATES AT ACQUALINA,9100.0,NaN,2024.0,NaN,2747.0,34638.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11764383,N,Y,25.942332,-80.120166,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Michael Goldstein,Acqualina Realty,None,None,Benjamin Moss,"Compass Florida, LLC.",Liz Hogan,"Compass Florida, LLC.",1,orange,ESTATES AT ACQUALINA at 17975 Collins Ave Unit...
1,PAST SALE,2025-03-11,Condo/Co-op,125 Ocean Dr,Miami Beach,FL,33139.0,24000000.0,4.0,5.0,OCEAN HOUSE CONDO,3936.0,NaN,2009.0,NaN,6098.0,10001.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/125-Ocea...,MARMLS,A11738003,N,Y,25.770080,-80.132744,https://www.redfin.com/FL/Miami-Beach/125-Ocea...,Sarah Williams,Societe Real Estate,None,None,Michael Duchon,The Corcoran Group,Eloy Carmenate,The Corcoran Group,2,blue,OCEAN HOUSE CONDO at 125 Ocean Dr in Miami Beach
2,PAST SALE,2025-03-11,Condo/Co-op,7264 Fisher Is #7264,Fisher Island,FL,33109.0,10900000.0,3.0,3.5,Oceanside,4590.0,NaN,1992.0,NaN,2375.0,9738.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7264-Fis...,MARMLS,A11653700,N,Y,25.762287,-80.134708,https://www.redfin.com/FL/Miami-Beach/7264-Fis...,Lazaro Rivero,"Rivero Real Estate , Inc.",Rose Bauer,"Rivero Real Estate , Inc.",Lazaro Rivero,"Rivero Real Estate , Inc.",None,None,3,blue,Oceanside at 7264 Fisher Is #7264 in Fisher Is...
3,PAST SALE,2025-03-12,Condo/Co-op,18501 Collins Ave #801,Sunny Isles Beach,FL,33160.0,6000000.0,4.0,5.5,Turnberry Ocean Club,3252.0,NaN,2020.0,NaN,1845.0,6063.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Beaches MLS,RX-10968656,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Heidi Brzyski,Douglas Elliman (Jupiter),Tracy Servideo,Douglas Elliman (Jupiter),Julio Oliveira,Elite International Realty Inc,None,None,4,blue,Turnberry Ocean Club at 18501 Collins Ave #801...
4,PAST SALE,2025-03-10,Condo/Co-op,15701 Collins Ave #3602,Sunny Isles Beach,FL,33160.0,5445000.0,3.0,3.5,15701 COLLINS CONDO,2475.0,NaN,2020.0,NaN,2200.0,3640.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11690661,N,Y,25.921944,-80.121716,https://www.redfin.com/FL/North-Miami-Beach/15...,Kelly Salinas,RE/MAX Advance Realty II,None,None,Dennis Mangone,Douglas Elliman,None,None,5,blue,15701 COLLINS CONDO at 15701 Collins Ave #3602...
5,PAST SALE,2025-03-11,Condo/Co-op,6800 Indian Creek Dr Unit 7A,Miami Beach,FL,33141.0,4800000.0,3.0,3.5,AMD PL OF 2ND OCEAN FRONT,2844.0,NaN,2022.0,NaN,1688.0,6414.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6800-Ind...,MARMLS,A11471887,N,Y,25.852444,-80.122990,https://www.redfin.com/FL/Miami-Beach/6800-Ind...,Nelson Gonzalez,BHHS EWM Realty,None,None,Arno De Vos,De Vos Real Estate,Paul Basile,Douglas Elliman,6,blue,AMD PL OF 2ND OCEAN FRONT at 6800 Indian Creek...
6,PAST SALE,2025-03-10,Condo/Co-op,6000 Island Blvd #2604,Aventura,FL,33160.0,4775000.0,5.0,4.5,BELLAMARE @ WILLIAMS ISLA,4221.0,NaN,2005.0,NaN,1131.0,4608.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/6000-Island...,MARMLS,A11699721,N,Y,25.942761,-80.135433,https://www.redfin.com/FL/Aventura/6000-Island...,Richard Goihman,Douglas Elliman,Ivonne Goihman,Douglas Elliman,Richard Goihman,Douglas Elliman,None,None,7,blue,BELLAMARE @ WILLIAMS ISLA at 6000 Island Blvd ...
7,PAST SALE,2025-03-10,Condo/Co-op,17875 Collins Ave #4401,Sunny Isles Beach,FL,33160.0,4350000.0,4.0,4.0,ACQUALINA OCEAN RESIDENCE,2736

In [55]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: ESTATES AT ACQUALINA closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $3,500,000 to $25,000,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami Beach, Fisher Island, Sunny Isles Beach, Aventura, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 135 condo sales totaling $177,194,410 million from Mar. 9th to Mar. 16th. The previous week, brokers closed 117 condo sales totaling $120,844,976.

Last week’s units sold for an average of $1,312,551, higher than the $1,032,863 average pri

In [56]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Sunny-Isles-Beach/17975-Collins-Ave-33160/unit-4701-2/home/193479101


In [57]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami-Beach/125-Ocean-Dr-33139/home/194263734


In [58]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami-Beach/50-S-Pointe-Dr-33139/unit-1207/home/43388314


In [59]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Sunny-Isles-Beach/16901-Collins-Ave-33160/unit-2602/home/56742200


In [63]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami-Beach/125-Ocean-Dr-33139/home/194263734


In [64]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Aventura/6000-Island-Blvd-33160/unit-2604/home/43342622


In [65]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-03-14,Condo/Co-op,17975 Collins Ave Unit 4701/2,Sunny Isles Beach,FL,33160.0,25000000.0,6.0,7.0,ESTATES AT ACQUALINA,9100.0,NaN,2024.0,NaN,2747.0,34638.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11764383,N,Y,25.942332,-80.120166,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Michael Goldstein,Acqualina Realty,None,None,Benjamin Moss,"Compass Florida, LLC.",Liz Hogan,"Compass Florida, LLC.",1,orange,ESTATES AT ACQUALINA at 17975 Collins Ave Unit...
1,PAST SALE,2025-03-11,Condo/Co-op,125 Ocean Dr,Miami Beach,FL,33139.0,24000000.0,4.0,5.0,OCEAN HOUSE CONDO,3936.0,NaN,2009.0,NaN,6098.0,10001.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/125-Ocea...,MARMLS,A11738003,N,Y,25.770080,-80.132744,https://www.redfin.com/FL/Miami-Beach/125-Ocea...,Sarah Williams,Societe Real Estate,None,None,Michael Duchon,The Corcoran Group,Eloy Carmenate,The Corcoran Group,2,blue,OCEAN HOUSE CONDO at 125 Ocean Dr in Miami Beach
2,PAST SALE,2025-03-11,Condo/Co-op,7264 Fisher Is #7264,Fisher Island,FL,33109.0,10900000.0,3.0,3.5,Oceanside,4590.0,NaN,1992.0,NaN,2375.0,9738.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7264-Fis...,MARMLS,A11653700,N,Y,25.762287,-80.134708,https://www.redfin.com/FL/Miami-Beach/7264-Fis...,Lazaro Rivero,"Rivero Real Estate , Inc.",Rose Bauer,"Rivero Real Estate , Inc.",Lazaro Rivero,"Rivero Real Estate , Inc.",None,None,3,blue,Oceanside at 7264 Fisher Is #7264 in Fisher Is...
3,PAST SALE,2025-03-12,Condo/Co-op,18501 Collins Ave #801,Sunny Isles Beach,FL,33160.0,6000000.0,4.0,5.5,Turnberry Ocean Club,3252.0,NaN,2020.0,NaN,1845.0,6063.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Beaches MLS,RX-10968656,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Heidi Brzyski,Douglas Elliman (Jupiter),Tracy Servideo,Douglas Elliman (Jupiter),Julio Oliveira,Elite International Realty Inc,None,None,4,blue,Turnberry Ocean Club at 18501 Collins Ave #801...
4,PAST SALE,2025-03-10,Condo/Co-op,15701 Collins Ave #3602,Sunny Isles Beach,FL,33160.0,5445000.0,3.0,3.5,15701 COLLINS CONDO,2475.0,NaN,2020.0,NaN,2200.0,3640.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11690661,N,Y,25.921944,-80.121716,https://www.redfin.com/FL/North-Miami-Beach/15...,Kelly Salinas,RE/MAX Advance Realty II,None,None,Dennis Mangone,Douglas Elliman,None,None,5,blue,15701 COLLINS CONDO at 15701 Collins Ave #3602...
5,PAST SALE,2025-03-11,Condo/Co-op,6800 Indian Creek Dr Unit 7A,Miami Beach,FL,33141.0,4800000.0,3.0,3.5,AMD PL OF 2ND OCEAN FRONT,2844.0,NaN,2022.0,NaN,1688.0,6414.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6800-Ind...,MARMLS,A11471887,N,Y,25.852444,-80.122990,https://www.redfin.com/FL/Miami-Beach/6800-Ind...,Nelson Gonzalez,BHHS EWM Realty,None,None,Arno De Vos,De Vos Real Estate,Paul Basile,Douglas Elliman,6,blue,AMD PL OF 2ND OCEAN FRONT at 6800 Indian Creek...
6,PAST SALE,2025-03-10,Condo/Co-op,6000 Island Blvd #2604,Aventura,FL,33160.0,4775000.0,5.0,4.5,BELLAMARE @ WILLIAMS ISLA,4221.0,NaN,2005.0,NaN,1131.0,4608.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/6000-Island...,MARMLS,A11699721,N,Y,25.942761,-80.135433,https://www.redfin.com/FL/Aventura/6000-Island...,Richard Goihman,Douglas Elliman,Ivonne Goihman,Douglas Elliman,Richard Goihman,Douglas Elliman,None,None,7,blue,BELLAMARE @ WILLIAMS ISLA at 6000 Island Blvd ...
7,PAST SALE,2025-03-10,Condo/Co-op,17875 Collins Ave #4401,Sunny Isles Beach,FL,33160.0,4350000.0,4.0,4.0,ACQUALINA OCEAN RESIDENCE,2736

## Time on Market Calculator

In [79]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 11, 11) ## List (Earlier) date
date2 = datetime(2025, 3, 14) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

123


## Clean CSV for Datawrapper Chart

In [68]:
chart_df = df_top_ten

In [69]:
chart_df = chart_df.fillna(" ")

In [70]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [71]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [72]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [73]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [74]:
chart_df['$/SQUARE FEET'].astype(float)

0    2747.0
1    6098.0
2    2375.0
3    1845.0
4    2200.0
5    1688.0
6    1131.0
7    1590.0
8    1373.0
9    2426.0
Name: $/SQUARE FEET, dtype: float64

In [75]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [76]:
csv_date_string = today.strftime("%m_%d_%Y")

In [77]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [78]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-03-14,17975 Collins Ave Unit 4701/2 in Sunny Isles B...,"$25,000,000",6,7.0,"9,100",2024,"$2,747",MARMLS,A11764383,Michael Goldstein Acqualina Realty,"Benjamin Moss Compass Florida, LLC. Liz Hogan ..."
1,2025-03-11,125 Ocean Dr in Miami Beach,"$24,000,000",4,5.0,"3,936",2009,"$6,098",MARMLS,A11738003,Sarah Williams Societe Real Estate,Michael Duchon The Corcoran Group Eloy Carmena...
2,2025-03-11,7264 Fisher Is #7264 in Fisher Island,"$10,900,000",3,3.5,"4,590",1992,"$2,375",MARMLS,A11653700,"Lazaro Rivero Rivero Real Estate , Inc. Rose B...","Lazaro Rivero Rivero Real Estate , Inc."
3,2025-03-12,18501 Collins Ave #801 in Sunny Isles Beach,"$6,000,000",4,5.5,"3,252",2020,"$1,845",Beaches MLS,RX-10968656,Heidi Brzyski Douglas Elliman (Jupiter) Tracy ...,Julio Oliveira Elite International Realty Inc
4,2025-03-10,15701 Collins Ave #3602 in Sunny Isles Beach,"$5,445,000",3,3.5,"2,475",2020,"$2,200",MARMLS,A11690661,Kelly Salinas RE/MAX Advance Realty II,Dennis Mangone Douglas Elliman
5,2025-03-11,6800 Indian Creek Dr Unit 7A in Miami Beach,"$4,800,000",3,3.5,"2,844",2022,"$1,688",MARMLS,A11471887,Nelson Gonzalez BHHS EWM Realty,Arno De Vos De Vos Real Estate Paul Basile Dou...
6,2025-03-10,6000 Island Blvd #2604 in Aventura,"$4,775,000",5,4.5,"4,221",2005,"$1,131",MARMLS,A11699721,Richard Goihman Douglas Elliman Ivonne Goihman...,Richard Goihman Douglas Elliman
7,2025-03-10,17875 Collins Ave #4401 in Sunny Isles Beach,"$4,350,000",4,4.0,"2,736",2006,"$1,590",MARMLS,A11699457,Yansy Checa Douglas Elliman,Monica Villate EXP Realty LLC
8,2025-03-12,16901 Collins Ave #2602 in Sunny Isles Beach,"$4,000,000",3,4.5,"2,913",2018,"$1,373",MARMLS,A11651859,Rosa Maria Pinneri LPT Realty Kinga Konsorska ...,Wictor Arraes Xcellence Realty
9,2025-03-13,50 S Pointe Dr #1207 in Miami Beach,"$3,500,000",2,2.5,"1,443",2008,"$2,426",MARMLS,A11553824,Lisa Yanowitz Luxe Realty Group LLC,Sonia Toth Brown Harris Stevens
